In [1]:
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.models import Model

tmp = np.load('E:\\INSOFE\\AI and Deep Learning\\DAE_FGen\\cifar_pca_train.npz')
train_data = tmp['data']
train_labels = tmp['labels']

tmp = np.load('E:\\INSOFE\\AI and Deep Learning\\DAE_FGen\\cifar_pca_test.npz')
test_data = tmp['data']
test_labels = tmp['labels']

Using TensorFlow backend.


In [2]:
# Training an autoencoder model on cifar-10 PCA reduced data

input_img = Input(shape=(781,))
crrpt_img = Dropout(0.5)(input_img)
encoded = Dense(1000, activation='sigmoid')(crrpt_img)
decoded = Dense(781, activation='linear')(encoded)

autoencoder = Model(input_img,decoded)

nb_epoch = 50
batch_size = 32

autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error',
                    metrics=['mse'])


In [3]:
history = autoencoder.fit(train_data, train_data,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_data, test_data),
                    verbose=1)


autoencoder.save('DAE_l1_model.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 96s - loss: 0.5831 - mean_squared_error: 0.5831 - val_loss: 0.2351 - val_mean_squared_error: 0.2351
Epoch 2/50
50000/50000 [==============================] - 84s - loss: 0.5380 - mean_squared_error: 0.5380 - val_loss: 0.2127 - val_mean_squared_error: 0.2127
Epoch 3/50
50000/50000 [==============================] - 83s - loss: 0.5296 - mean_squared_error: 0.5296 - val_loss: 0.2049 - val_mean_squared_error: 0.2049
Epoch 4/50
50000/50000 [==============================] - 90s - loss: 0.5285 - mean_squared_error: 0.5285 - val_loss: 0.2029 - val_mean_squared_error: 0.2029
Epoch 5/50
50000/50000 [==============================] - 88s - loss: 0.5257 - mean_squared_error: 0.5257 - val_loss: 0.2023 - val_mean_squared_error: 0.2023
Epoch 6/50
50000/50000 [==============================] - 92s - loss: 0.5253 - mean_squared_error: 0.5253 - val_loss: 0.1984 - val_mean_squared_error: 0.1984
Ep

In [4]:
from keras.models import load_model
autoencoder = load_model("E:\\INSOFE\\AI and Deep Learning\\DAE_FGen\\DAE_l1_model.h5")

In [5]:
encoder = Model(input_img,encoded)
htrain_data = encoder.predict(train_data)


In [9]:
print(train_data.shape)
print(htrain_data.shape)

(50000, 781)
(50000, 1000)


In [14]:
print(train_labels)
print(test_labels)

[6 9 9 ..., 9 1 1]
[3 8 8 ..., 5 1 7]


In [15]:
# Converting labels into one-hot vectors for MLP training

train_labels = keras.utils.to_categorical(train_labels,10)
test_labels = keras.utils.to_categorical(test_labels,10)

In [ ]:
# training a two hidden layer MLP for classification task

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(781,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = mlp.fit(train_data[:10000], train_labels[:10000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(test_data, test_labels))

In [ ]:
# training an MLP on autoencoder features

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(1000,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

htest_data = encoder.predict(test_data)
history = mlp.fit(htrain_data[:10000], train_labels[:10000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(htest_data, test_labels))